In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import numpy as np

For reproducibility and consistency across runs, we will set a seed

In [57]:
def set_random_seeds(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        
set_random_seeds()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(torch.cuda.get_device_name(0))  # Prints the GPU name

Using device: cuda
NVIDIA GeForce RTX 4060 Laptop GPU


In [58]:
def load_mnist_data(batch_size=64, download=True):
    """
    Loads and returns MNIST train and test DataLoaders.
    
    Args:
        batch_size (int): The batch size for the DataLoader.
        download (bool): Whether to download the dataset if not found.
    
    Returns:
        train_loader, test_loader: DataLoader objects for the MNIST dataset.
    """
    # Define a transform to normalize the data
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    # Load the training and test datasets
    train_dataset = datasets.MNIST(root='../datasets', train=True, transform=transform, download=download) #'..' to refer back to motherfolder
    test_dataset = datasets.MNIST(root='../datasets', train=False, transform=transform, download=download)
    
    # Create DataLoaders for the datasets
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader

# Load the MNIST data
train_loader, test_loader = load_mnist_data()

In [59]:
class Autoencoder(nn.Module):
    """
    Autoencoder with a bottleneck of size 2 that maps MNIST images to a 2D latent space.
    Includes training, evaluation, and embedding extraction methods.
    """
    
    ### NOTES ###
    # Might have to use batchnorm to impose a structure on the latent space

    def __init__(self, latent_dim=2, hidden_size=128, use_batchnorm=True):
        super().__init__()
        # Encoder layers
        # 784 -> 128 -> 2
        encoder_layers = [
            nn.Flatten(),
            nn.Linear(28 * 28, hidden_size), # asuming size 28x28 of the images
            nn.ReLU()
        ]
        if use_batchnorm:
            encoder_layers.append(nn.Batchnorm1d(hidden_size))
        encoder_layers.append(nn.Linear(hidden_size, latent_dim)) # the size 2 bottleneck layer
        self.encoder = nn.Sequential(*encoder_layers) # '*' is unpacking the list into it's elements

        # Decoder layers
        # 2 -> 128 -> 784
        decoder_layers = [
            nn.Linear(latent_dim, hidden_size),
            nn.ReLU()
        ]
        if use_batchnorm:
            decoder_layers.append(nn.Batchnorm1d(hidden_size))
        decoder_layers.extend([
            nn.Linear(hidden_size, 28 * 28),
            nn.Sigmoid() # normalize outputs to [0, 1] - grayscale
        ])

        self.decoder = nn.Sequential(*decoder_layers)

    def encode(self, x):
        """
        Encodes an input batch (e.g., MNIST images) into the latent space.
        
        Args:
            x (Tensor): Input images of shape [batch_size, 1, 28, 28].
        Returns:
            z (Tensor): Encoded latent vectors of shape [batch_size, latent_dim].
        """

        z = self.encoder(x)
        return z


    def decode(self, z):
        """
        Decodes latent vectors back to the original image space.
        
        Args:
            z (Tensor): Latent vectors of shape [batch_size, latent_dim].
        Returns:
            x_rec (Tensor): Reconstructed images of shape [batch_size, 1, 28, 28].
        """

        x_rec = self.decoder(z)
        return x_rec

    def forward(self, x):
        """
        Complete forward pass: encode then decode.
        
        Args:
            x (Tensor): Input images.
        Returns:
            reconstructed (Tensor): Reconstructed images of the same shape as x.
        """
        reconstructed = self.decode(self.encode(x))
        return reconstructed

    def train_one_epoch(self, train_loader, optimizer, criterion, device='cpu'):
        """
        Performs one epoch of training.
        
        Args:
            train_loader (DataLoader): DataLoader for the training set.
            optimizer (torch.optim.Optimizer): Optimizer for model parameters.
            criterion: Loss function (e.g., MSELoss, BCELoss).
            device (str): 'cpu' or 'cuda' device.
        
        Returns:
            epoch_loss (float): Average loss across this training epoch.
        """
        loss_total = 0.0
        self.train()

        for x, _ in train_loader:
            x = x.to(device)
            # Computing loss
            reconstructed = self.forward(x)
            loss = criterion(reconstructed, x)
            loss_total += loss.item()
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        epoch_loss = loss_total / len(train_loader) # Computing average loss in epoch
        return epoch_loss

    def evaluate(self, data_loader, criterion, device='cpu'):
        """
        Evaluates the autoencoder on a given dataset (test or validation).
        
        Args:
            data_loader (DataLoader): DataLoader for the evaluation set.
            criterion: Loss function for reconstruction.
            device (str): 'cpu' or 'cuda'.
        
        Returns:
            eval_loss (float): Average reconstruction loss on this dataset.
        """
        self.eval() # Disable gradient computation

        loss_total = 0.0
        for x, _ in data_loader:
            x = x.to(device)
            # Computing loss
            reconstructed = self.forward(x)
            loss = criterion(reconstructed, x)
            loss_total += loss.item()

        eval_loss = loss_total / len(data_loader) # Computing average evaluation loss
        return eval_loss

    def fit(self, train_loader, test_loader, num_epochs, lr=1e-3, device='cpu', verbose=True):
        """
        High-level method to train the autoencoder for a given number of epochs.
        It orchestrates optimizer setup, training loop, and evaluation per epoch.
        
        Args:
            train_loader (DataLoader): DataLoader for training set.
            test_loader (DataLoader): DataLoader for test/validation set.
            num_epochs (int): Number of epochs.
            lr (float): Learning rate for the optimizer.
            device (str): 'cpu' or 'cuda'.
            verbose (bool): Each epoch prints loss if True
        
        Returns:
            train_losses (list of float): Loss for each training epoch.
            test_losses (list of float): Loss for each test epoch.
        """
        self.to(device)
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        loss_function = nn.BCELoss()

        train_loss_list = []
        test_loss_list = []
        # Fitting the model
        for epoch in range(num_epochs):
            # Train loss
            train_loss = self.train_one_epoch(train_loader, optimizer, criterion=loss_function,device=device)
            train_loss_list.append(train_loss)
            # Test loss
            test_loss = self.evaluate(test_loader, criterion=loss_function, device=device)
            if verbose:
                print(f'Epoch #{epoch}')
                print(f'Train Loss = {train_loss:.3e} --- Test Loss = {test_loss:.3e}')
        
        return train_loss_list, test_loss_list

    def get_latent_embeddings(self, data_loader, device='cpu'):
        """
        Passes the entire dataset through the encoder to extract latent vectors.
        
        Args:
            data_loader (DataLoader): DataLoader for the dataset to encode.
            device (str): 'cpu' or 'cuda'.
        
        Returns:
            embeddings (Tensor): Concatenated latent vectors of shape [N, latent_dim].
            labels (Tensor): Corresponding labels (if available) of shape [N].
        """
        
        embeddings = []
        labels = []

        self.eval() # Disable gradient computation

        for x, label in data_loader:
            x.to(device)
            z = self.encode(x)
            embeddings.append(z)
            labels.append(label)
        
        # Concatenate the list to tensors for consistency
        embeddings_concat = torch.cat(embeddings, dim=0)
        labels_concat  = torch.cat(labels, dim=0)

        return embeddings_concat, labels_concat


In [67]:
def select_anchors(embeddings, num_anchors=10):
    """
    Selects a subset of embeddings to use as 'anchors' for relative representation.
    
    Args:
        embeddings (Tensor or array): Shape [N, latent_dim].
        num_anchors (int): Number of anchors to select.
    
    Returns:
        anchors: A (num_anchors, latent_dim) subset of the original embeddings.
    """
    anchors_index = np.random.choice(embeddings.shape[0], size=num_anchors, replace=False)
    anchors = embeddings[anchors_index]
    return anchors

def compute_relative_coordinates(embeddings, anchors, flatten=False):
    """
    Transforms 'embeddings' into a 'relative' coordinate system based on anchors.
    One approach could be subtracting an anchor or computing offsets.
    
    Args:
        embeddings (Tensor): Shape [N, latent_dim].
        anchors (Tensor): Shape [A, latent_dim], where A = num_anchors.
    
    Returns:
        relative_embeds (Tensor): The embeddings expressed relative to the anchors.
    """
    # we start off with normalizing the embeddings an the anchors
    # this way the cosine similarity is just the dot product between the word and anchor
    embeddings_norm = embeddings/np.linalg.norm(embeddings, axis=1)[:, np.newaxis]
    anchors_norm = anchors/np.linalg.norm(anchors, axis=1)[:, np.newaxis]
    reletive_reps = []
    for embedding in embeddings_norm:
        if flatten:
            embedding = embedding.flatten()
        
        reletive_rep = np.array([np.dot(embedding, anchor) for anchor in anchors_norm])
        reletive_reps.append(reletive_rep)
    reletive_reps = np.array(reletive_reps)
    return reletive_reps


In [75]:
def run_experiment(num_epochs=5, batch_size=64, lr=1e-3, device='cpu', anchors_num=2):
    """
    Orchestrates the autoencoder pipeline:
      1. Load data
      2. Initialize the autoencoder
      3. Train and evaluate
      4. Extract embeddings
      5. (Optional) Select anchors, compute relative coordinates
    
    Args:
        num_epochs (int): Number of training epochs.
        batch_size (int): DataLoader batch size.
        lr (float): Learning rate.
        device (str): 'cpu' or 'cuda' device.
        latent_dim (int): Dimension of the AE's latent space (2 for easy visualization).
    
    Returns:
        model: Trained autoencoder.
        embeddings (Tensor): Latent embeddings from the test (or train) set.
        anchors (Tensor): (Optional) set of anchor embeddings if you implement that step here.
    """
    # create the data loader
    train_loader, test_loader = load_mnist_data()

    # extact the data to select anchors
    all_features = []

    for features, labels in train_loader:
        # Move to CPU if they are on GPU (if you're using CUDA)
        features = features.cpu()
        
        all_features.append(features)

    # Concatenate all batches into a single tensor
    all_features = torch.cat(all_features, dim=0)

    # select anchors
    anchors = select_anchors(all_features, num_anchors=anchors_num)
    print("anchor shape is ", anchors[0].shape)

    AE = Autoencoder
    """
    Train the encoder
    """

    encoded_data, encoded_anchors = AE.encode(all_features).numpy(), AE.encode(anchors).numpy()
    rel_reps = compute_relative_coordinates(encoded_data, encoded_anchors)
    return rel_reps
run_experiment()

torch.Size([1, 28, 28])
